In [1]:
# Code for Capstone Final Project - T. LaFrance
# Project explores location of alcohol and liquor establishments vs crimes in DFW area

# There are TWO elements of this analysis - one uses Foursquare to retrieve location data for the local 
# area mentioned.  The other pulls data from a Dallas County crime portal.  Both datasources are 
# exploratory in relating alcohol establishment to local crime


In [8]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

In [20]:
CLIENT_ID = 'GGQBZIJ4FVXH1CEYFZWK5PZGKJS10HLYBKJZKFUYXBCT55AC' # your Foursquare ID
CLIENT_SECRET = 'NNNH2C4Z22R5QQRPYMRMR0XMLGYC2GCSJPV01QZ50CYWPGN5' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GGQBZIJ4FVXH1CEYFZWK5PZGKJS10HLYBKJZKFUYXBCT55AC
CLIENT_SECRET:NNNH2C4Z22R5QQRPYMRMR0XMLGYC2GCSJPV01QZ50CYWPGN5


In [99]:
# Point of origin selected as Los Angeles Union Station, near downtown LA, 10 mile radius
latitude = 34.05
longitude = -118.2
radius = 10
LIMIT = 100

# Describe search
search_query = 'Nightclubs'
print(search_query + ' .... OK!')


Nightclubs .... OK!


In [100]:


# create URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5eeac6a59da7ee001bd4050a'},
 'response': {'venues': [{'id': '4dab63cb93a04642f09502c4',
    'name': 'Little Mexico',
    'location': {'lat': 34.0479935169538,
     'lng': -118.20024207775364,
     'labeledLatLngs': [{'label': 'display',
       'lat': 34.0479935169538,
       'lng': -118.20024207775364}],
     'distance': 224,
     'postalCode': '90033',
     'cc': 'US',
     'city': 'Los Angeles',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['Los Angeles, CA 90033', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1c1941735',
      'name': 'Mexican Restaurant',
      'pluralName': 'Mexican Restaurants',
      'shortName': 'Mexican',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1592444604',
    'hasPerk': False},
   {'id': '5ba5fdd5b37e2b002c3f6b5b',
    'name': 'Metro by T-Mobile',
    'location': 

In [101]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,False,4dab63cb93a04642f09502c4,NaN,...,224,"[Los Angeles, CA 90033, United States]","[{'label': 'display', 'lat': 34.0479935169538,...",34.047994,-118.200242,90033,CA,Little Mexico,v-1592444604,NaN
1,"[{'id': '4f04afc02fb6e1c99f3db0bc', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,False,5ba5fdd5b37e2b002c3f6b5b,915 Forest Ave,...,139,"[915 Forest Ave, Los Angeles, CA 90033, United...","[{'label': 'display', 'lat': 34.05022257902103...",34.050223,-118.198511,90033,CA,Metro by T-Mobile,v-1592444604,NaN
2,"[{'id': '4bf58dd8d48988d1fc941735', 'name': 'L...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c82b49374d7b60c434183d8,911 Forest Ave,...,150,"[911 Forest Ave, Los Angeles, CA 90033, United...","[{'label': 'display', 'lat': 34.05034255981445...",34.050343,-118.198418,90033,CA,Lavanderia Laundry,v-1592444604,NaN
3,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,False,59d7a8b986f4cc7f7d7b8937,923 Forest Ave,...,164,"[923 Forest Ave, Los Angeles, CA 90033, United...","[{'label': 'display', 'lat': 34.0504193409921,...",34.050419,-118.198289,90033,CA,X'tiosu Kitchen,v-1592444604,NaN
4,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c4622e5dd35be9a8af9dcd8,2801 Wabash Ave,...,224,"[2801 Wabash Ave, Los Angeles, CA 90033, Unite...","[{'label': 'display', 'lat': 34.05077999999999...",34.050780,-118.197761,90033,CA,Los Angeles Public Library - Malabar,v-1592444604,NaN


In [102]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
 
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Little Mexico,Mexican Restaurant,NaN,US,Los Angeles,United States,NaN,224,"[Los Angeles, CA 90033, United States]","[{'label': 'display', 'lat': 34.0479935169538,...",34.047994,-118.200242,90033,CA,4dab63cb93a04642f09502c4
1,Metro by T-Mobile,Mobile Phone Shop,915 Forest Ave,US,Los Angeles,United States,NaN,139,"[915 Forest Ave, Los Angeles, CA 90033, United...","[{'label': 'display', 'lat': 34.05022257902103...",34.050223,-118.198511,90033,CA,5ba5fdd5b37e2b002c3f6b5b
2,Lavanderia Laundry,Laundry Service,911 Forest Ave,US,Los Angeles,United States,NaN,150,"[911 Forest Ave, Los Angeles, CA 90033, United...","[{'label': 'display', 'lat': 34.05034255981445...",34.050343,-118.198418,90033,CA,4c82b49374d7b60c434183d8
3,X'tiosu Kitchen,Mexican Restaurant,923 Forest Ave,US,Los Angeles,United States,NaN,164,"[923 Forest Ave, Los Angeles, CA 90033, United...","[{'label': 'display', 'lat': 34.0504193409921,...",34.050419,-118.198289,90033,CA,59d7a8b986f4cc7f7d7b8937
4,Los Angeles Public Library - Malabar,Library,2801 Wabash Ave,US,Los Angeles,United States,NaN,224,"[2801 Wabash Ave, Los Angeles, CA 90033, Unite...","[{'label': 'display', 'lat': 34.05077999999999...",34.050780,-118.197761,90033,CA,4c4622e5dd35be9a8af9dcd8
5,Dennys Market,Food,2730 Wabash Ave,US,Los Angeles,United States,NaN,114,"[2730 Wabash Ave, Los Angeles, CA 90033, Unite...","[{'label': 'display', 'lat': 34.050761, 'lng':...",34.050761,-118.199161,90033,CA,4f32419d19836c91c7c5a3a9
6,ARCO,Gas Station,2925 E Cesar Ave,US,Los Angeles,United States,NaN,669,"[2925 E Cesar Ave, Los Angeles, CA 90033, Unit...","[{'label': 'display', 'lat': 34.044, 'lng': -1...",34.044000,-118.200541,90033,CA,5b54dd740d173f002c5a15a1
7,Los Angeles Cardiovascular Consultants Medical...,Building,11645 Wilshire Blvd Ste 825,US,Los Angeles,United States,NaN,685,"[11645 Wilshire Blvd Ste 825, Los Angeles, CA ...","[{'label': 'display', 'lat': 34.05614315, 'lng...",34.056143,-118.200473,90025,CA,4dadd175cda1aece4a61c418
8,Metro 620 Boyle Heights Shuttle,Moving Target,NaN,US,Los Angeles,United States,NaN,290,"[Los Angeles, CA 90033, United States]","[{'label': 'display', 'lat': 34.04998016357422...",34.049980,-118.196854,90033,CA,50bcfa61e4b0382138ab80fe
9,Family Store Outlet,Miscellaneous Shop,2810 E Cesar E Chavez Ave,US,Los Angeles,United States,NaN,671,"[2810 E Cesar E Chavez Ave, Los Angeles, CA 90...","[{'label': 'display', 'lat': 34.04436111450195...",34.044361,-118.202583,90033,CA,4da0dc37bf22a14302388ef8


In [ ]:
# the next section creates visuallization of venues data from foursquare

In [103]:
# generate map centred around Los Angeles Union Station (near downtown LA)

venues_map = folium.Map(location=[latitude, longitude], zoom_start=14) 

# add a red circle marker to represent LA Union Station
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Los Angeles Union Station',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add blue circle markers to represent local venues on map
for lat, lng, name, categories in zip(dataframe_filtered ['lat'], dataframe_filtered['lng'], dataframe_filtered['name'], dataframe_filtered['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map) 

# display map
venues_map

In [81]:
# This is the start of the next portion where Los Angeles crime info is explored

# Reading City of LA Open Portal Data with local crime info

df_la = pd.read_json('https://data.lacity.org/resource/63jg-8b9z.json')
df_la.head(10)

,area,area_name,crm_cd,crm_cd_1,crm_cd_2,crm_cd_desc,cross_street,date_occ,date_rptd,dr_no,...,premis_desc,rpt_dist_no,status,status_desc,time_occ,vict_age,vict_descent,vict_sex,weapon_desc,weapon_used_cd
0,13,Newton,900,900,NaN,VIOLATION OF COURT ORDER,NaN,2010-02-20T00:00:00.000,2010-02-20T00:00:00.000,1307355,...,SINGLE FAMILY DWELLING,1385,AA,Adult Arrest,1350,48,H,M,NaN,NaN
1,14,Pacific,740,740,NaN,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",MANCHESTER AV,2010-09-12T00:00:00.000,2010-09-13T00:00:00.000,11401303,...,STREET,1485,IC,Invest Cont,45,0,W,M,NaN,NaN
2,13,Newton,946,946,NaN,OTHER MISCELLANEOUS CRIME,NaN,2010-08-09T00:00:00.000,2010-08-09T00:00:00.000,70309629,...,ALLEY,1324,IC,Invest Cont,1515,0,H,M,NaN,NaN
3,6,Hollywood,900,900,998.0,VIOLATION OF COURT ORDER,HOLLYWOOD BL,2010-01-05T00:00:00.000,2010-01-05T00:00:00.000,90631215,...,STREET,646,IC,Invest Cont,150,47,W,F,HAND GUN,102.0
4,1,Central,122,122,NaN,"RAPE, ATTEMPTED",SAN PEDRO ST,2010-01-02T00:00:00.000,2010-01-03T00:00:00.000,100100501,...,ALLEY,176,IC,Invest Cont,2100,47,H,F,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400.0
5,1,Central,442,442,NaN,SHOPLIFTING - PETTY THEFT ($950 & UNDER),NaN,2010-01-04T00:00:00.000,2010-01-05T00:00:00.000,100100506,...,DEPARTMENT STORE,162,AA,Adult Arrest,1650,23,B,M,NaN,NaN
6,1,Central,330,330,NaN,BURGLARY FROM VEHICLE,GRAND AV,2010-01-07T00:00:00.000,2010-01-08T00:00:00.000,100100508,...,STREET,182,IC,Invest Cont,2005,46,H,M,NaN,NaN
7,1,Central,230,230,NaN,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",NaN,2010-01-08T00:00:00.000,2010-01-09T00:00:00.000,100100509,...,OTHER PREMISE,157,AA,Adult Arrest,2100,51,B,M,UNKNOWN WEAPON/OTHER WEAPON,500.0
8,1,Central,230,230,NaN,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",NaN,2010-01-09T00:00:00.000,2010-01-09T00:00:00.000,100100510,...,PARKING LOT,171,IC,Invest Cont,230,30,H,M,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400.0
9,1,Central,341,341,998.0,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",NaN,2010-01-06T00:00:00.000,2010-01-09T00:00:00.000,100100511,...,OTHER PREMISE,132,IC,Invest Cont,2100,55,W,M,NaN,NaN


In [85]:
#Filter data to remove NaN in location fields; 

df_la = df_la[["lat","lon","crm_cd_desc","date_occ"]]

df_la.drop(df_la[df_la['lat'] == 'NaN'].index, inplace = True)
df_la.drop(df_la[df_la['lon'] == 'NaN'].index, inplace = True)

#simplify / clean data a little more to just relevant info

df_la.head(10)


,lat,lon,crm_cd_desc,date_occ
0,33.9825,-118.2695,VIOLATION OF COURT ORDER,2010-02-20T00:00:00.000
1,33.9599,-118.3962,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",2010-09-12T00:00:00.000
2,34.0224,-118.2524,OTHER MISCELLANEOUS CRIME,2010-08-09T00:00:00.000
3,34.1016,-118.3295,VIOLATION OF COURT ORDER,2010-01-05T00:00:00.000
4,34.0387,-118.2488,"RAPE, ATTEMPTED",2010-01-02T00:00:00.000
5,34.0480,-118.2577,SHOPLIFTING - PETTY THEFT ($950 & UNDER),2010-01-04T00:00:00.000
6,34.0389,-118.2643,BURGLARY FROM VEHICLE,2010-01-07T00:00:00.000
7,34.0435,-118.2427,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",2010-01-08T00:00:00.000
8,34.0450,-118.2640,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",2010-01-09T00:00:00.000
9,34.0538,-118.2488,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",2010-01-06T00:00:00.000


In [98]:
# generate CRIME MAP
# generate map centred around LA Union Station

crime_map = folium.Map(location=[34.05, -118.25], zoom_start=14) 

# add a red circle marker to represent Los Angeles Union Station
folium.features.CircleMarker(
    [34.05, -118.25],
    radius=10,
    color='red',
    popup='Los Angeles Union Station',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(crime_map)

# add CRIME markers to map
for lat, lng, crm_cd_desc in zip(df_la['lat'], df_la['lon'], df_la['crm_cd_desc']):
    label = '{}'.format(crm_cd_desc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(crime_map)  

# display map
crime_map

In [ ]:
# hover over a point to see the description of the crime
# from the visualiztion, it seems that the crime data tends to be reported in general blocks, (ie. 3400 block of broadway, etc.) 
# as the points appear to fall gridlike